In [2]:
import pandas as pd
import os
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report, f1_score, roc_auc_score
import pickle
import Functions2 as ef
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, accuracy_score
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


**Bad data**

In [6]:
# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(ef.measure_pigment_network(original))
            feature_2.append(ef.measure_blue_veil(original))
            feature_3.append(ef.measure_vascular(original))
            feature_4.append(ef.measure_globules(original))
            feature_5.append(ef.measure_streaks(original))
            feature_6.append(ef.measure_irregular_pigmentation(original))
            feature_7.append(ef.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Training'

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Merge the features DataFrame with the diagnostic column from the original DataFrame
df_merged2 = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

# Split the data into training and testing sets
X = df_merged2.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged2['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()

KeyboardInterrupt: 

**KNN MODEL AND PICKLE**

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from sklearn.decomposition import PCA

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform PCA
pca = PCA(0.95)  # Adjust the number of components as per your requirement
X_pca = pca.fit_transform(X)

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X_pca, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()

**Predict with Knn**

In [11]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import Functions2 as ef
from sklearn.neighbors import KNeighborsClassifier

# Load the pickled models
knn_model_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl"  # Update with the actual path to the KNN model
lr_mixed_model_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_mixed_model.pkl"  # Update with the actual path to the lr_mixed model
lr_good_model_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl"  # Update with the actual path to the lr_good model

with open(knn_model_path, "rb") as f:
    knn_model = pickle.load(f)

with open(lr_mixed_model_path, "rb") as f:
    lr_mixed_model = pickle.load(f)

with open(lr_good_model_path, "rb") as f:
    lr_good_model = pickle.load(f)

# Open the CSV file with diagnostic information
data_path = r"C:\Users\serru\Downloads\archive (1)\GroundTruth.csv"  # Update with the actual path to the ground truth CSV file
df1 = pd.read_csv(data_path)

# Create a new DataFrame with image ID and diagnostic columns
new_df = pd.DataFrame()
new_df['img_id'] = df1['image']
new_df['diagnostic'] = df1[['MEL', 'BCC', 'AKIEC']].any(axis=1).astype(int)
df1 = new_df

# Load the data with diagnostic labels
data_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv"  # Update with the actual path to the full data CSV file
df2 = pd.read_csv(data_path)

# Preprocess the diagnostic column
df2['diagnostic'] = df2['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Concatenate the two dataframes
df = pd.concat([df1, df2], ignore_index=True)

# Remove file extensions from image IDs
df['img_id'] = df['img_id'].str.replace('.png', '')
df['img_id'] = df['img_id'].str.replace('.jpg', '')

# Define the function to extract features
def extract_features(image_path):
    try:
        image = cv2.imread(image_path)
        lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

        # Perform feature extraction operations
        feature_1 = ef.measure_pigment_network(lab_image)
        feature_2 = ef.measure_blue_veil(lab_image)
        feature_3 = ef.measure_vascular(lab_image)
        feature_4 = ef.measure_globules(lab_image)
        feature_5 = ef.measure_streaks(lab_image)
        feature_6 = ef.measure_irregular_pigmentation(lab_image)
        feature_7 = ef.measure_regression(lab_image)

        return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None

# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\testMixed"  # Update with the actual path to the folder containing mixed test images

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels_knn = []
predicted_labels_lr_mixed = []
predicted_labels_lr_good = []

# Iterate over files in the folder
for filename in os.listdir(folder_path_in):
    image_path = os.path.join(folder_path_in, filename)
    
    # Check if the path is a file
    if os.path.isfile(image_path):
        # Extract features from the image
        features = extract_features(image_path)
        
        # Proceed if features were successfully extracted
        if features is not None:
            # Prepare the data for prediction
            X = np.array(features).reshape(1, -1)

            # Make predictions using the KNN model
            predicted_label_knn = knn_model.predict(X)
            predicted_labels_knn.append(predicted_label_knn)

            # Make predictions using the lr_mixed model
            probabilities_lr_mixed = lr_mixed_model.predict_proba(X)
            predicted_label_lr_mixed = probabilities_lr_mixed[0][1]  # Assuming the second class represents the positive class
            predicted_labels_lr_mixed.append(predicted_label_lr_mixed)

            # Make predictions using the lr_good model
            probabilities_lr_good = lr_good_model.predict_proba(X)
            predicted_label_lr_good = probabilities_lr_good[0][1]  # Assuming the second class represents the positive class
            predicted_labels_lr_good.append(predicted_label_lr_good)

            # Extract the image ID from the filename without the file extension
            image_id = os.path.splitext(filename)[0]

            # Retrieve the true label from the DataFrame if the image ID exists
            if image_id in df['img_id'].values:
                true_label = df[df['img_id'] == image_id]['diagnostic'].values[0]
                true_labels.append(true_label)
            else:
                print(f"No matching image ID found for {filename}")

# Convert the lists to numpy arrays
Y = np.array(true_labels)
y_pred_knn = np.array(predicted_labels_knn)
y_pred_lr_mixed = np.array(predicted_labels_lr_mixed)
y_pred_lr_good = np.array(predicted_labels_lr_good)

# Convert predicted probabilities to binary predictions
threshold = 0.5  # Update the threshold if necessary
y_pred_lr_mixed = (y_pred_lr_mixed > threshold).astype(int)
y_pred_lr_good = (y_pred_lr_good > threshold).astype(int)

# Calculate evaluation metrics for KNN
f1_knn = f1_score(Y, y_pred_knn)
precision_knn = precision_score(Y, y_pred_knn)
recall_knn = recall_score(Y, y_pred_knn)
tn_knn, fp_knn, fn_knn, tp_knn = confusion_matrix(Y, y_pred_knn).ravel()
specificity_knn = tn_knn / (tn_knn + fp_knn)

# Calculate evaluation metrics for lr_mixed
f1_lr_mixed = f1_score(Y, y_pred_lr_mixed)
precision_lr_mixed = precision_score(Y, y_pred_lr_mixed)
recall_lr_mixed = recall_score(Y, y_pred_lr_mixed)
tn_lr_mixed, fp_lr_mixed, fn_lr_mixed, tp_lr_mixed = confusion_matrix(Y, y_pred_lr_mixed).ravel()
specificity_lr_mixed = tn_lr_mixed / (tn_lr_mixed + fp_lr_mixed)

# Calculate evaluation metrics for lr_good
f1_lr_good = f1_score(Y, y_pred_lr_good)
precision_lr_good = precision_score(Y, y_pred_lr_good)
recall_lr_good = recall_score(Y, y_pred_lr_good)
tn_lr_good, fp_lr_good, fn_lr_good, tp_lr_good = confusion_matrix(Y, y_pred_lr_good).ravel()
specificity_lr_good = tn_lr_good / (tn_lr_good + fp_lr_good)

# Display the evaluation metrics for KNN
print("K-Nearest Neighbors Model:")
print(f"F1 score: {f1_knn}")
print(f"Precision: {precision_knn}")
print(f"Recall/Sensitivity: {recall_knn}")
print(f"Specificity: {specificity_knn}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred_knn))
print()

# Display the evaluation metrics for lr_mixed
print("Logistic Regression (lr_mixed) Model:")
print(f"F1 score: {f1_lr_mixed}")
print(f"Precision: {precision_lr_mixed}")
print(f"Recall/Sensitivity: {recall_lr_mixed}")
print(f"Specificity: {specificity_lr_mixed}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred_lr_mixed))
print()

# Display the evaluation metrics for lr_good
print("Logistic Regression (lr_good) Model:")
print(f"F1 score: {f1_lr_good}")
print(f"Precision: {precision_lr_good}")
print(f"Recall/Sensitivity: {recall_lr_good}")
print(f"Specificity: {specificity_lr_good}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred_lr_good))
print()


c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\s

Error processing C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\testMixed\mixed.csv: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\s

K-Nearest Neighbors Model:
F1 score: 0.5365853658536585
Precision: 0.36666666666666664
Recall/Sensitivity: 1.0
Specificity: 0.09523809523809523
Confusion Matrix:
[[ 2 19]
 [ 0 11]]

Logistic Regression (lr_mixed) Model:
F1 score: 0.41379310344827586
Precision: 0.3333333333333333
Recall/Sensitivity: 0.5454545454545454
Specificity: 0.42857142857142855
Confusion Matrix:
[[ 9 12]
 [ 5  6]]

Logistic Regression (lr_good) Model:
F1 score: 0.5365853658536585
Precision: 0.36666666666666664
Recall/Sensitivity: 1.0
Specificity: 0.09523809523809523
Confusion Matrix:
[[ 2 19]
 [ 0 11]]



c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\serru\.conda\envs\New\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [3]:
# Iterate over files in the folder
for filename in os.listdir(folder_path_in):
    file_path = os.path.join(folder_path_in, filename)
    
    # Check if the path is a file
    if os.path.isfile(file_path):
        try:
            # Extract features from the image
            features = extract_features(file_path)

            # Prepare the data for prediction
            X = np.array(features).reshape(1, -1)

            # Make predictions using the loaded Lr model
            probabilities = model.predict_proba(X)
            predicted_label = probabilities[0][1]  # Assuming the second class represents the positive class
            predicted_labels.append(predicted_label)

            # Extract the image ID from the filename without the file extension
            image_id = os.path.splitext(filename)[0]

            # Retrieve the true label from the DataFrame if the image ID exists
            if image_id in df['img_id'].values:
                true_label = df[df['img_id'] == image_id]['diagnostic'].values[0]
                true_labels.append(true_label)
            else:
                print(f"No matching image ID found for {filename}")
        except cv2.error as e:
            print(f"Error processing {filename}: {str(e)}")


NameError: name 'folder_path_in' is not defined

In [50]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import Functions2 as ef


predicted_labels = []
true_labels = []

folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\testMixed"  # Update with the actual path to the folder containing images
db_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\testMixed\mixed.csv"  # Update with the actual path to mixed.csv
model_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl"

# Load the database
df = pd.read_csv(db_path)

# Load the trained model
model = pickle.load(open(model_path, "rb"))

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    print(f"Processing file: {filename}")
    image_path = os.path.join(folder_path_in, filename)

    # Extract features from the image
    features = extract_features(image_path)

    if features is None:
        print(f"Failed to extract features for {filename}. Skipping...")
        continue

    # Prepare the data for prediction
    X = np.array(features).reshape(1, -1)

    # Extract the first 8 characters from the filename
    img_id = filename[:8]

    # Check if the image ID exists in the dataframe
    if img_id in df['img_id'].values:
        print(f"Making prediction for {filename}...")
        # Make predictions using the loaded model
        probabilities = model.predict_proba(X)
        predicted_label = probabilities[0][1]  # Assuming the second class represents the positive class
        predicted_labels.append(predicted_label)

        # Retrieve the true label from the dataframe
        true_label = df[df['img_id'] == img_id]['diagnostic'].values[0]
        true_labels.append(true_label)
    else:
        print(f"No matching image ID found for {filename}. Skipping...")

# After the loop over images, check if there are any predictions made
if len(predicted_labels) > 0:
    # Convert predicted labels to binary labels
    y_pred = np.array(predicted_labels) > 0.5  # Threshold at 0.5 to classify as positive or negative
    y_pred = y_pred.astype(int)  # Convert boolean array to integer array

    y_true = np.array(true_labels)

    # Calculate evaluation metrics and confusion matrix
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)

    # Calculate specificity (True Negative Rate)
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    specificity = TN / (TN + FP)

    # Print the results
    print(f"Classifier: {type(model).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion)
else:
    print("No predictions made.")


Processing file: ISIC_0034294.jpg
Failed to extract features for ISIC_0034294.jpg. Skipping...
Processing file: ISIC_0034295.jpg
Failed to extract features for ISIC_0034295.jpg. Skipping...
Processing file: ISIC_0034296.jpg
Failed to extract features for ISIC_0034296.jpg. Skipping...
Processing file: ISIC_0034297.jpg
Failed to extract features for ISIC_0034297.jpg. Skipping...
Processing file: ISIC_0034298.jpg
Failed to extract features for ISIC_0034298.jpg. Skipping...
Processing file: ISIC_0034301.jpg
Failed to extract features for ISIC_0034301.jpg. Skipping...
Processing file: ISIC_0034302.jpg
Failed to extract features for ISIC_0034302.jpg. Skipping...
Processing file: ISIC_0034303.jpg
Failed to extract features for ISIC_0034303.jpg. Skipping...
Processing file: ISIC_0034304.jpg
Failed to extract features for ISIC_0034304.jpg. Skipping...
Processing file: ISIC_0034305.jpg
Failed to extract features for ISIC_0034305.jpg. Skipping...
Processing file: mixed.csv
Failed to extract featu

In [46]:
feature_2

16926

In [26]:
import numpy as np
from sklearn.decomposition import PCA

# Initialize an empty list to store the features
features = []

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path_in, filename)

        # Extract features from the image
        feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features2(image_path)

        # Append the features to the list
        features.append([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7])

# Convert the list of features to a NumPy array
X = np.array(features)

# Perform PCA
pca = PCA(n_components=7)  # Specify the number of components you want to keep
X_pca = pca.fit_transform(X)

# Access the principal components
principal_components = pca.components_

# Access the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Access the singular values (eigenvalues)
singular_values = pca.singular_values_

# Print the results
print("Principal Components:")
print(principal_components)
print("Explained Variance Ratio:")
print(explained_variance_ratio)
print("Singular Values:")
print(singular_values)

import pickle

# Perform PCA and obtain the principal components
pca = PCA(n_components=2)  # Specify the desired number of components
X_pca = pca.fit_transform(X)

# Save the PCA model using pickle
pca_model_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\pca_model.pkl"
with open(pca_model_path, "wb") as f:
    pickle.dump(pca, f)

print("PCA model saved successfully.")

Principal Components:
[[-1.76743736e-04  9.90577944e-01 -2.88732902e-03 -3.00325489e-04
   2.06317980e-37  6.36382631e-04  1.36917760e-01]
 [ 5.40017917e-04 -1.36898746e-01  5.21193385e-03  4.91566580e-03
   1.50747125e-35  6.43725911e-04  9.90558781e-01]
 [-1.30989074e-02  3.61920352e-03  9.97414625e-01  6.85612041e-02
   2.35770029e-33  1.59005231e-02 -5.09124811e-03]
 [ 9.95806854e-01  2.38035122e-04  1.93609196e-02 -8.85272609e-02
  -1.38777878e-17 -1.25181737e-02 -1.64398083e-04]
 [ 8.37571969e-02  3.42621950e-04 -6.58402390e-02  8.56300661e-01
   3.33066907e-16  5.05352318e-01 -4.22970069e-03]
 [ 3.43734959e-02  8.92541590e-04 -2.04641871e-02  5.04169822e-01
  -1.11022302e-16 -8.62675306e-01 -1.72904469e-03]
 [ 0.00000000e+00  7.79423986e-20 -1.02372487e-17 -9.52847371e-18
   1.00000000e+00 -2.21603598e-16  2.29269136e-19]]
Explained Variance Ratio:
[9.59851347e-01 3.97791760e-02 3.44091384e-04 1.34642069e-05
 1.13632038e-05 5.57779853e-07 8.67635361e-38]
Singular Values:
[3.3222

**PCA TEST**

In [ ]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.decomposition import PCA

# Load the pickled KNN model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\pcaknn.pkl", "rb") as f:
    model = pickle.load(f)

# Load the data
data_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv"  # Update with the actual path to mixed.csv
df = pd.read_csv(data_path)

# Create a new DataFrame for modified data
new_df = pd.DataFrame()
new_df['img_id'] = df['    image']
new_df['diagnostic'] = df[['MEL', 'BCC', 'AKIEC']].any(axis=1).astype(int)
df = new_df

# Define the function to extract features
def extract_features2(image_path):
    image = cv2.imread(image_path)
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Perform feature extraction operations
    feature_1 = ef.measure_pigment_network(lab_image)
    feature_2 = ef.measure_blue_veil(lab_image)
    feature_3 = ef.measure_vascular(lab_image)
    feature_4 = ef.measure_globules(lab_image)
    feature_5 = ef.measure_streaks(lab_image)
    feature_6 = ef.measure_irregular_pigmentation(lab_image)
    feature_7 = ef.measure_regression(lab_image)

    # Return the features as a single array
    return np.array([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7])

# Define the folder path for image processing
folder_path_in = r"CC:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\TestBad"  # Update with the actual path to the folder containing images

# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Initialize PCA
pca = PCA(n_components=7)  # Choose the desired number of components

# Prepare the data for PCA fitting
X_train = []

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path_in, filename)

        # Extract features from the image
        feature_vector = extract_features2(image_path)

        # Append the feature vector to the training data
        X_train.append(feature_vector)

# Fit PCA on the training data
X_train = np.array(X_train)
pca.fit(X_train)

# Iterate over images in the folder again to make predictions
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path_in, filename)

        # Extract features from the image
        feature_vector = extract_features2(image_path)

        # Apply PCA transformation
        feature_vector_pca = pca.transform(feature_vector.reshape(1, -1))

        # Prepare the data for prediction
        X = feature_vector_pca.reshape(1, -1)

        # Make predictions using the loaded KNN model
        probabilities = model.predict_proba(X)

        # Get the probability for class 1 (positive class)
        probability_positive = probabilities[0][1]

        # Map the probability to a value between 0 and 1
        mapped_value = 1.0 - probability_positive  # Invert the probability to get the desired mapping

        # Get the diagnosis from the new_df DataFrame using the modified filename
        matching_rows = new_df[new_df['img_id'].str.contains(filename)]
        if not matching_rows.empty:
            diagnosis = matching_rows['diagnostic'].values[0]
            # Map the diagnosis to a human-readable label
            diagnosis_label = 1 if diagnosis == 1 else 0
        else:
            diagnosis_label = 'Unknown'

        # Append true label and predicted label to the respective lists
        true_labels.append(int(diagnosis))
        predicted_labels.append(mapped_value)

# Print explained variance ratio
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# After the loop over images, convert predicted labels to binary labels
y_pred = np.array(predicted_labels) > 0.5  # Threshold at 0.5 to classify as positive or negative
y_pred = y_pred.astype(int)  # Convert boolean array to integer array

y_true = np.array(true_labels)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
confusion = confusion_matrix(y_true, y_pred)

# Calculate specificity (True Negative Rate)
TN = confusion[0, 0]
FP = confusion[0, 1]
specificity = TN / (TN + FP)

print(f"Classifier: {type(model).__name__}")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall/Sensitivity: {recall}")
print(f"Specificity: {specificity}")
print("Confusion Matrix:")
print(confusion)


**Good Data**

In [ ]:
# Load the data
data_path = r"C:\Users\serru\Downloads\archive (1)\helpme\newdata.csv"
df = pd.read_csv(data_path)

new_df = pd.DataFrame()
new_df['img_id'] = df['image']
new_df['diagnostic'] = df[['MEL', 'BCC', 'AKIEC']].any(axis=1).astype(int)
df = new_df

# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(ef.measure_pigment_network(original))
            feature_2.append(ef.measure_blue_veil(original))
            feature_3.append(ef.measure_vascular(original))
            feature_4.append(ef.measure_globules(original))
            feature_5.append(ef.measure_streaks(original))
            feature_6.append(ef.measure_irregular_pigmentation(original))
            feature_7.append(ef.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r'C:\Users\serru\Downloads\archive (1)\helpme\both_resized'

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7


features_df['img_id'] = features_df['img_id'].str.replace('.jpg', '')

df_merged3 = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

if df_merged3.shape[0] == 0:
    print("Dataset is empty!")
else:
    # Check the column names
    if 'img_id' not in df_merged3.columns or 'diagnostic' not in df_merged3.columns:
        print("Column names not found in the dataset!")
    else:
        X = df_merged3.drop(['img_id', 'diagnostic'], axis=1)
        Y = df_merged3['diagnostic']

        # Adjust test_size based on the size of your dataset
        test_size = 0.2  # 20% of the data will be used for testing

        # Split the data into training and test sets
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)

        # Continue with the rest of your code
        classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()


**LR MODEL AND PICKLE**

In [ ]:
import numpy as np
import pickle
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

# Create LR model
lr_model = LogisticRegression()
# Train the LR model on your data (X and Y)
lr_model.fit(X, Y)

# Pickle the LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl", "wb") as f:
    pickle.dump(lr_model, f)

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl", "rb") as f:
    loaded_lr_model = pickle.load(f)

# Perform cross-validation using the loaded LR model
y_pred = cross_val_predict(loaded_lr_model, X, Y, cv=5)

# Calculate evaluation metrics
f1 = f1_score(Y, y_pred)
precision = precision_score(Y, y_pred)
recall = recall_score(Y, y_pred)
tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
specificity = tn / (tn + fp)

print("Loaded LR Model Metrics:")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall/Sensitivity: {recall}")
print(f"Specificity: {specificity}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred))

**Predict with LR good**

In [ ]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\PCAKNN.pkl", "rb") as f:
    model = pickle.load(f)

# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features2(image_path):
    image = cv2.imread(image_path)
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Perform feature extraction operations
    feature_1 = ef.measure_pigment_network(lab_image)
    feature_2 = ef.measure_blue_veil(lab_image)
    feature_3 = ef.measure_vascular(lab_image)
    feature_4 = ef.measure_globules(lab_image)
    feature_5 = ef.measure_streaks(lab_image)
    feature_6 = ef.measure_irregular_pigmentation(lab_image)
    feature_7 = ef.measure_regression(lab_image)

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"  # Replace with the path to the folder containing images

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Update the file extensions if necessary
        image_path = os.path.join(folder_path_in, filename)
        
        # Extract features from the image
        feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features2(image_path)
        
        # Prepare the data for prediction
        X = np.array([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7]).reshape(1, -1)
        
        # Make predictions using the loaded LR model
        probabilities = model.predict_proba(X)
        
        # Get the probability for class 1 (positive class)
        probability_positive = probabilities[0][1]
        
        # Map the probability to a value between 0 and 1
        mapped_value = 1.0 - probability_positive  # Invert the probability to get the desired mapping

        # Get the diagnosis from the df DataFrame using the modified filename
        diagnosis = df.loc[df['img_id'] == filename, 'diagnostic'].values[0]

        # Map the diagnosis to a human-readable label
        diagnosis_label = 'Not Healthy' if diagnosis == 1 else 'Healthy'
        
        # Print the image filename, diagnosis, and mapped value
        print("Image:", filename)
        print("Diagnosis from csv:", diagnosis_label)
        print("Prediction [0:Healthy - 1:Not Healthy]:", mapped_value)
        print("---------------------")


**Mixed data**

In [ ]:
df_merged4 = pd.concat([df_merged3, df_merged2], ignore_index=True)
df_merged4

#save the new dataset
df_merged4.to_csv(r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\mixed.csv', index=False)

In [ ]:
# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(ef.measure_pigment_network(original))
            feature_2.append(ef.measure_blue_veil(original))
            feature_3.append(ef.measure_vascular(original))
            feature_4.append(ef.measure_globules(original))
            feature_5.append(ef.measure_streaks(original))
            feature_6.append(ef.measure_irregular_pigmentation(original))
            feature_7.append(ef.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r'C:\Users\serru\Downloads\archive (1)\mixed'

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Split the data into training and testing sets
X = df_merged4.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged4['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()

**AND PICKLE**

In [ ]:
import numpy as np
import pickle
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

# Create LR model
model = LogisticRegression()  # Specify the number of neighbors

# Train the LR model on your data (X and Y)
model.fit(X, Y)

# Pickle the LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_mixed_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_mixed_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Perform cross-validation using the loaded KNN model
y_pred = cross_val_predict(loaded_lr_model, X, Y, cv=5)

# Calculate evaluation metrics
f1 = f1_score(Y, y_pred)
precision = precision_score(Y, y_pred)
recall = recall_score(Y, y_pred)
tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
specificity = tn / (tn + fp)

print("Loaded Mixed LR Model Metrics:")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall/Sensitivity: {recall}")
print(f"Specificity: {specificity}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred))

**Predict with LR mixed**

In [ ]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd

# Load the pickled LR model
model_path = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\PCAKNN.pkl"
if not os.path.isfile(model_path):
    print("Error: Model file not found.")
    exit()
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
if not os.path.isfile(data_path):
    print("Error: Data file not found.")
    exit()
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features2(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to read image - {image_path}")
        return None
    
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Perform feature extraction operations
    feature_1 = ef.measure_pigment_network(lab_image)
    feature_2 = ef.measure_blue_veil(lab_image)
    feature_3 = ef.measure_vascular(lab_image)
    feature_4 = ef.measure_globules(lab_image)
    feature_5 = ef.measure_streaks(lab_image)
    feature_6 = ef.measure_irregular_pigmentation(lab_image)
    feature_7 = ef.measure_regression(lab_image)

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"  # Replace with the path to the folder containing images

# Check if the folder exists
if not os.path.isdir(folder_path_in):
    print("Error: Folder not found.")
    exit()

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Update the file extensions if necessary
        image_path = os.path.join(folder_path_in, filename)
        
        # Extract features from the image
        features = extract_features2(image_path)
        if features is None:
            continue
        
        # Prepare the data for prediction
        X = np.array(features).reshape(1, -1)
        
        # Make predictions using the loaded LR model
        probabilities = model.predict_proba(X)
        
        # Get the probability for class 1 (positive class)
        probability_positive = probabilities[0][1]
        
        # Map the probability to a value between 0 and 1
        mapped_value = 1.0 - probability_positive  # Invert the probability to get the desired mapping

        # Get the diagnosis from the df DataFrame using the modified filename
        diagnosis = df.loc[df['img_id'] == filename, 'diagnostic'].values
        if len(diagnosis) == 0:
            print(f"Error: Diagnosis not found for image - {filename}")
            continue
        diagnosis = diagnosis[0]

        # Map the diagnosis to a human-readable label
        diagnosis_label = 'Not Healthy' if diagnosis == 1 else 'Healthy'
        
        # Print the image filename, diagnosis, and mapped value
        print("Image:", filename)
        print("Diagnosis from csv:", diagnosis_label)
        print("Prediction [0:Healthy - 1:Not Healthy]:", mapped_value)
        print("---------------------")


In [ ]:
feature_1